In [ ]:
from causalinference import CausalModel
from causalinference.utils import random_data

Y, D, X = random_data()
# print(Y)
# print(D)
# print(X)
causal = CausalModel(Y, D, X)

In [ ]:
print(causal.summary_stats)

In [ ]:
causal.summary_stats['X_t_mean']

In [ ]:
causal.summary_stats.keys()

In [ ]:
causal.est_via_ols()
print(causal.estimates)

- ATE: average treatment effect
- ATC: average treatment effect for controls
- ATT: average treatment effect for treated

Constant treatment effect: $\textit{adj=1}$ in $\texttt{est_via_ols}$.

In [ ]:
causal.est_propensity_s()
print(causal.propensity)

In [ ]:
causal.propensity.keys()

In [ ]:
causal.cutoff

In [ ]:
causal.trim_s()
causal.cutoff

In [ ]:
print(causal.summary_stats)

In [ ]:
causal.stratify_s()
print(causal.strata)

In [ ]:
for stratum in causal.strata:
    stratum.est_via_ols(adj=1)

In [ ]:
[stratum.estimates['ols']['ate'] for stratum in causal.strata]

In [ ]:
causal.est_via_ols()
causal.est_via_weighting()
causal.est_via_blocking()
causal.est_via_matching(bias_adj=True)
print(causal.estimates)

In [ ]:
from pyprocessmacro import Process
import pandas as pd

df = pd.read_csv("./obsdat.csv")
df.head()

In [ ]:
p = Process(data=df, model=13, x="a", y="y", w="w1", z="z", m="m", logit=True)
p.summary()

In [ ]:
ind_model = p.indirect_model

In [ ]:
ind_model.coeff_summary()